In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install timm


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 431 kB 7.1 MB/s 


In [5]:
!pwd 
%cd drive/MyDrive/models/dino-main/

/content
/content/drive/MyDrive/models/dino-main


In [6]:
!nvidia-smi

Wed May 25 08:50:10 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Copyright (c) Facebook, Inc. and its affiliates.
# 
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     http://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
import os
import sys
import pickle
import argparse

import torch
from torch import nn
import torch.distributed as dist
import torch.backends.cudnn as cudnn
from torchvision import models as torchvision_models
from torchvision import transforms as pth_transforms
from PIL import Image, ImageFile
import numpy as np

import utils
import vision_transformer as vits
from eval_knn import extract_features


class OxfordParisDataset(torch.utils.data.Dataset):
    def __init__(self, dir_main, dataset, split, transform=None, imsize=None):
        if dataset not in ['roxford5k', 'rparis6k']:
            raise ValueError('Unknown dataset: {}!'.format(dataset))

        # loading imlist, qimlist, and gnd, in cfg as a dict
        gnd_fname = os.path.join(dir_main, dataset, 'gnd_{}.pkl'.format(dataset))
        with open(gnd_fname, 'rb') as f:
            cfg = pickle.load(f)
        cfg['gnd_fname'] = gnd_fname
        cfg['ext'] = '.jpg'
        cfg['qext'] = '.jpg'
        cfg['dir_data'] = os.path.join(dir_main, dataset)
        cfg['dir_images'] = os.path.join(cfg['dir_data'], 'jpg')
        cfg['n'] = len(cfg['imlist'])
        cfg['nq'] = len(cfg['qimlist'])
        cfg['im_fname'] = config_imname
        cfg['qim_fname'] = config_qimname
        cfg['dataset'] = dataset
        self.cfg = cfg

        self.samples = cfg["qimlist"] if split == "query" else cfg["imlist"]
        self.transform = transform
        self.imsize = imsize

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index):
        path = os.path.join(self.cfg["dir_images"], self.samples[index] + ".jpg")
        ImageFile.LOAD_TRUNCATED_IMAGES = True
        with open(path, 'rb') as f:
            img = Image.open(f)
            img = img.convert('RGB')
        if self.imsize is not None:
            img.thumbnail((self.imsize, self.imsize), Image.ANTIALIAS)
        if self.transform is not None:
            img = self.transform(img)
        return img, index


def config_imname(cfg, i):
    return os.path.join(cfg['dir_images'], cfg['imlist'][i] + cfg['ext'])


def config_qimname(cfg, i):
    return os.path.join(cfg['dir_images'], cfg['qimlist'][i] + cfg['qext'])


if __name__ == '__main__':
    parser = argparse.ArgumentParser('Image Retrieval on revisited Paris and Oxford')
    parser.add_argument('--data_path', default='/content/drive/MyDrive/dataSets/CRC/DINO-test-same/', type=str)
    parser.add_argument('--dataset', default='roxford5k', type=str, choices=['roxford5k', 'rparis6k'])
    parser.add_argument('--multiscale', default=False, type=utils.bool_flag)
    parser.add_argument('--imsize', default=224, type=int, help='Image size')
    parser.add_argument('--pretrained_weights', default='', type=str, help="Path to pretrained weights to evaluate.")
    parser.add_argument('--use_cuda', default=True, type=utils.bool_flag)
    parser.add_argument('--arch', default='vit_small', type=str, help='Architecture')
    parser.add_argument('--patch_size', default=16, type=int, help='Patch resolution of the model.')
    parser.add_argument("--checkpoint_key", default="teacher", type=str,
        help='Key to use in the checkpoint (example: "teacher")')
    parser.add_argument('--num_workers', default=10, type=int, help='Number of data loading workers per GPU.')
    parser.add_argument("--dist_url", default="env://", type=str, help="""url used to set up
        distributed training; see https://pytorch.org/docs/stable/distributed.html""")
    parser.add_argument("--local_rank", default=0, type=int, help="Please ignore and do not set this argument.")
    args = parser.parse_args("")

    utils.init_distributed_mode(args)
    print("git:\n  {}\n".format(utils.get_sha()))
    print("\n".join("%s: %s" % (k, str(v)) for k, v in sorted(dict(vars(args)).items())))
    cudnn.benchmark = True

    # ============ preparing data ... ============
    transform = pth_transforms.Compose([
        pth_transforms.ToTensor(),
        pth_transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])
    dataset_train = OxfordParisDataset(args.data_path, args.dataset, split="train", transform=transform, imsize=args.imsize)
    dataset_query = OxfordParisDataset(args.data_path, args.dataset, split="query", transform=transform, imsize=args.imsize)
    sampler = torch.utils.data.DistributedSampler(dataset_train, shuffle=False)
    data_loader_train = torch.utils.data.DataLoader(
        dataset_train,
        sampler=sampler,
        batch_size=1,
        num_workers=args.num_workers,
        pin_memory=True,
        drop_last=False,
    )
    data_loader_query = torch.utils.data.DataLoader(
        dataset_query,
        batch_size=1,
        num_workers=args.num_workers,
        pin_memory=True,
        drop_last=False,
    )
    print(f"train: {len(dataset_train)} imgs / query: {len(dataset_query)} imgs")

    # ============ building network ... ============
    if "vit" in args.arch:
        model = vits.__dict__[args.arch](patch_size=args.patch_size, num_classes=0)
        print(f"Model {args.arch} {args.patch_size}x{args.patch_size} built.")
    elif "xcit" in args.arch:
        model = torch.hub.load('facebookresearch/xcit:main', args.arch, num_classes=0)
    elif args.arch in torchvision_models.__dict__.keys():
        model = torchvision_models.__dict__[args.arch](num_classes=0)
    else:
        print(f"Architecture {args.arch} non supported")
        sys.exit(1)
    if args.use_cuda:
        model.cuda()
    model.eval()

    # load pretrained weights
    if os.path.isfile(args.pretrained_weights):
        state_dict = torch.load(args.pretrained_weights, map_location="cpu")
        if args.checkpoint_key is not None and args.checkpoint_key in state_dict:
            print(f"Take key {args.checkpoint_key} in provided checkpoint dict")
            state_dict = state_dict[args.checkpoint_key]
        # remove `module.` prefix
        state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}
        # remove `backbone.` prefix induced by multicrop wrapper
        state_dict = {k.replace("backbone.", ""): v for k, v in state_dict.items()}
        msg = model.load_state_dict(state_dict, strict=False)
        print('Pretrained weights found at {} and loaded with msg: {}'.format(args.pretrained_weights, msg))
    elif args.arch == "vit_small" and args.patch_size == 16:
        print("Since no pretrained weights have been provided, we load pretrained DINO weights on Google Landmark v2.")
        model.load_state_dict(torch.hub.load_state_dict_from_url(url="https://dl.fbaipublicfiles.com/dino/dino_vitsmall16_googlelandmark_pretrain/dino_vitsmall16_googlelandmark_pretrain.pth"))
    else:
        print("Warning: We use random weights.")

    ############################################################################
    # Step 1: extract features
    train_features = extract_features(model, data_loader_train, args.use_cuda, multiscale=args.multiscale)
    query_features = extract_features(model, data_loader_query, args.use_cuda, multiscale=args.multiscale)

    if utils.get_rank() == 0:  # only rank 0 will work from now on
        # normalize features
        train_features = nn.functional.normalize(train_features, dim=1, p=2)
        query_features = nn.functional.normalize(query_features, dim=1, p=2)

        ############################################################################
        # Step 2: similarity
        sim = torch.mm(train_features, query_features.T)
        ranks = torch.argsort(-sim, dim=0).cpu().numpy()

        ############################################################################
        # Step 3: evaluate
        gnd = dataset_train.cfg['gnd']
        # evaluate ranks
        ks = [1, 5, 10]
        # search for easy & hard
        gnd_t = []
        for i in range(len(gnd)):
            g = {}
            g['ok'] = np.concatenate([gnd[i]['easy'], gnd[i]['hard']])
            g['junk'] = np.concatenate([gnd[i]['junk']])
            gnd_t.append(g)
        mapM, apsM, mprM, prsM = utils.compute_map(ranks, gnd_t, ks)
        # search for hard
        gnd_t = []
        for i in range(len(gnd)):
            g = {}
            g['ok'] = np.concatenate([gnd[i]['hard']])
            g['junk'] = np.concatenate([gnd[i]['junk'], gnd[i]['easy']])
            gnd_t.append(g)
        mapH, apsH, mprH, prsH = utils.compute_map(ranks, gnd_t, ks)
        print('>> {}: mAP M: {}, H: {}'.format(args.dataset, np.around(mapM*100, decimals=2), np.around(mapH*100, decimals=2)))
        print('>> {}: mP@k{} M: {}, H: {}'.format(args.dataset, np.array(ks), np.around(mprM*100, decimals=2), np.around(mprH*100, decimals=2)))
    dist.barrier()

In [ ]:
# Copyright (c) Facebook, Inc. and its affiliates.
# 
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     http://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
import os
import sys
import pickle
import argparse

import torch
from torch import nn
import torch.distributed as dist
import torch.backends.cudnn as cudnn
from torchvision import models as torchvision_models
from torchvision import transforms as pth_transforms
from PIL import Image, ImageFile
import numpy as np
from dataset import SingleData
import utils
from utils2 import  LabelSmoothingCrossEntropyLoss, BatchHardTripletLoss, ImageReader, MPerClassSampler
from torchvision import datasets, transforms
import vision_transformer as vits
from eval_knn import extract_features
import math


def get_data_list(data_path, ratio=0.001):
    img_list = []
    for root, dirs, files in os.walk(data_path):
        if files == []:
            class_name = dirs
        elif dirs == []:
            for f in files:
                img_path = os.path.join(root, f)
                img_list.append(img_path)

    np.random.seed(1)
    train_img_list = np.random.choice(img_list, size=int(len(img_list)*(1-ratio)), replace=False)
    #print(img_list, train_img_list)
    eval_img_list = list(set(img_list) - set(train_img_list))
    ########add
    half=math.floor(len(eval_img_list)/2)
    print(half)
    eval_=eval_img_list[:half]
    test_=eval_img_list[half:]
    #######
    #return class_name, train_img_list, eval_img_list 
    return class_name, train_img_list

class OxfordParisDataset(torch.utils.data.Dataset):
    def __init__(self, dir_main, dataset, split, transform=None, imsize=None):
        if dataset not in ['roxford5k', 'rparis6k']:
            raise ValueError('Unknown dataset: {}!'.format(dataset))

        # loading imlist, qimlist, and gnd, in cfg as a dict
        gnd_fname = os.path.join(dir_main, dataset, 'gnd_{}.pkl'.format(dataset))
        with open(gnd_fname, 'rb') as f:
            cfg = pickle.load(f)
        cfg['gnd_fname'] = gnd_fname
        cfg['ext'] = '.jpg'
        cfg['qext'] = '.jpg'
        cfg['dir_data'] = os.path.join(dir_main, dataset)
        cfg['dir_images'] = os.path.join(cfg['dir_data'], 'jpg')
        cfg['n'] = len(cfg['imlist'])
        cfg['nq'] = len(cfg['qimlist'])
        cfg['im_fname'] = config_imname
        cfg['qim_fname'] = config_qimname
        cfg['dataset'] = dataset
        self.cfg = cfg

        self.samples = cfg["qimlist"] if split == "query" else cfg["imlist"]
        self.transform = transform
        self.imsize = imsize

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index):
        path = os.path.join(self.cfg["dir_images"], self.samples[index] + ".jpg")
        ImageFile.LOAD_TRUNCATED_IMAGES = True
        with open(path, 'rb') as f:
            img = Image.open(f)
            img = img.convert('RGB')
        if self.imsize is not None:
            img.thumbnail((self.imsize, self.imsize), Image.ANTIALIAS)
        if self.transform is not None:
            img = self.transform(img)
        return img, index


def config_imname(cfg, i):
    return os.path.join(cfg['dir_images'], cfg['imlist'][i] + cfg['ext'])


def config_qimname(cfg, i):
    return os.path.join(cfg['dir_images'], cfg['qimlist'][i] + cfg['qext'])


if __name__ == '__main__':
    parser = argparse.ArgumentParser('Image Retrieval on revisited Paris and Oxford')
    parser.add_argument('--data_path', default='/content/drive/MyDrive/dataSets/oxbuild_images', type=str)
    parser.add_argument('--dataset', default='roxford5k', type=str, choices=['roxford5k', 'rparis6k'])
    parser.add_argument('--multiscale', default=False, type=utils.bool_flag)
    parser.add_argument('--imsize', default=224, type=int, help='Image size')
    parser.add_argument('--pretrained_weights', default='/content/drive/MyDrive/models/dino-main/checkpoint.pth', type=str, help="Path to pretrained weights to evaluate.")
    parser.add_argument('--use_cuda', default=True, type=utils.bool_flag)
    parser.add_argument('--arch', default='vit_small', type=str, help='Architecture')
    parser.add_argument('--patch_size', default=16, type=int, help='Patch resolution of the model.')
    parser.add_argument("--checkpoint_key", default="teacher", type=str,
        help='Key to use in the checkpoint (example: "teacher")')
    parser.add_argument('--num_workers', default=8, type=int, help='Number of data loading workers per GPU.')
    parser.add_argument("--dist_url", default="env://", type=str, help="""url used to set up
        distributed training; see https://pytorch.org/docs/stable/distributed.html""")
    parser.add_argument("--local_rank", default=1, type=int, help="Please ignore and do not set this argument.")
    args = parser.parse_args("")

    utils.init_distributed_mode(args)
    print("git:\n  {}\n".format(utils.get_sha()))
    print("\n".join("%s: %s" % (k, str(v)) for k, v in sorted(dict(vars(args)).items())))
    cudnn.benchmark = True

    # ============ preparing data ... ============
    transform = pth_transforms.Compose([
        pth_transforms.ToTensor(),
        pth_transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])
    
    #################################################
    # data_path="/content/drive/MyDrive/dataSets/CRC/DINO-test-same/train"
    # data_path="D:\\models\\CGD-master\\dataset\\img"
    # test_path="/content/drive/MyDrive/dataSets/CRC/DINO-test-same/query"
    # class_name, train_img_list = get_data_list(data_path)
    # class_test_name, test_img_list = get_data_list(test_path)
    # train_data_set=SingleData(class_name, train_img_list, transform=transform)
    # train_sample = MPerClassSampler(train_data_set.label_list, batch_size=8)
    # data_loader_train = torch.utils.data.DataLoader(train_data_set, batch_sampler=train_sample, num_workers=8)
    # train_data_set = datasets.ImageFolder(data_path, transform=transform)
    # test_data_set = datasets.ImageFolder(test_path, transform=transform)
    # test_data_set = SingleData(class_test_name, test_img_list, transform=transform)
    # data_loader_query = torch.utils.data.DataLoader(test_data_set, batch_size=8, shuffle=False, num_workers=8)
    ##################################################
    
    dataset_train = OxfordParisDataset(args.data_path, args.dataset, split="train", transform=transform, imsize=args.imsize)
    dataset_query = OxfordParisDataset(args.data_path, args.dataset, split="query", transform=transform, imsize=args.imsize)
    # sampler = torch.utils.data.DistributedSampler(train_data_set, shuffle=False)
    sampler = torch.utils.data.DistributedSampler(dataset_train, shuffle=False)

    # sampler = torch.utils.data.Sampler(dataset_train)
    data_loader_train = torch.utils.data.DataLoader(
        dataset_train,
        # train_data_set,
        sampler=sampler,
        # sampler=train_sample,
        batch_size=1,
        num_workers=args.num_workers,
        pin_memory=True,
        drop_last=False,
    )
    data_loader_query = torch.utils.data.DataLoader(
        dataset_query,
        # test_data_set,
        batch_size=1,
        num_workers=args.num_workers,
        pin_memory=True,
        drop_last=False,
    )
    # print(f"train: {len(dataset_train)} imgs / query: {len(dataset_query)} imgs")

    # ============ building network ... ============
    if "vit" in args.arch:
        model = vits.__dict__[args.arch](patch_size=args.patch_size, num_classes=0)
        print(f"Model {args.arch} {args.patch_size}x{args.patch_size} built.")
    elif "xcit" in args.arch:
        model = torch.hub.load('facebookresearch/xcit:main', args.arch, num_classes=0)
    elif args.arch in torchvision_models.__dict__.keys():
        model = torchvision_models.__dict__[args.arch](num_classes=0)
    else:
        print(f"Architecture {args.arch} non supported")
        sys.exit(1)
    if args.use_cuda:
        model.cuda()
    model.eval()

    # load pretrained weights
    if os.path.isfile(args.pretrained_weights):
        state_dict = torch.load(args.pretrained_weights, map_location="cpu")
        if args.checkpoint_key is not None and args.checkpoint_key in state_dict:
            print(f"Take key {args.checkpoint_key} in provided checkpoint dict")
            state_dict = state_dict[args.checkpoint_key]
        # remove `module.` prefix
        state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}
        # remove `backbone.` prefix induced by multicrop wrapper
        state_dict = {k.replace("backbone.", ""): v for k, v in state_dict.items()}
        msg = model.load_state_dict(state_dict, strict=False)
        print('Pretrained weights found at {} and loaded with msg: {}'.format(args.pretrained_weights, msg))
    elif args.arch == "vit_small" and args.patch_size == 16:
        print("Since no pretrained weights have been provided, we load pretrained DINO weights on Google Landmark v2.")
        model.load_state_dict(torch.hub.load_state_dict_from_url(url="https://dl.fbaipublicfiles.com/dino/dino_vitsmall16_googlelandmark_pretrain/dino_vitsmall16_googlelandmark_pretrain.pth"))
    else:
        print("Warning: We use random weights.")

    ############################################################################
    # Step 1: extract features
    train_features = extract_features(model, data_loader_train, args.use_cuda, multiscale=args.multiscale)
    query_features = extract_features(model, data_loader_query, args.use_cuda, multiscale=args.multiscale)
    torch.save(train_features, 'train_features.pt')
    torch.save(query_features, 'query_features.pt')
    print(train_features)
    print(query_features)
    if utils.get_rank() == 0:  # only rank 0 will work from now on
        # normalize features
        # train_features = nn.functional.normalize(train_features, dim=1, p=2)
        # query_features = nn.functional.normalize(query_features, dim=1, p=2)

        ############################################################################
        # Step 2: similarity
        sim = torch.mm(train_features, query_features.T)
        ranks = torch.argsort(-sim, dim=0).cpu().numpy()
        torch.save(sim, 'sim.pt')
        print(sim)

        print(ranks)

    #     ############################################################################
    #     # Step 3: evaluate
    #     gnd = dataset_train.cfg['gnd']
    #     # evaluate ranks
    #     ks = [1, 5, 10]
    #     # search for easy & hard
    #     gnd_t = []
    #     for i in range(len(gnd)):
    #         g = {}
    #         g['ok'] = np.concatenate([gnd[i]['easy'], gnd[i]['hard']])
    #         g['junk'] = np.concatenate([gnd[i]['junk']])
    #         gnd_t.append(g)
    #     mapM, apsM, mprM, prsM = utils.compute_map(ranks, gnd_t, ks)
    #     # search for hard
    #     gnd_t = []
    #     for i in range(len(gnd)):
    #         g = {}
    #         g['ok'] = np.concatenate([gnd[i]['hard']])
    #         g['junk'] = np.concatenate([gnd[i]['junk'], gnd[i]['easy']])
    #         gnd_t.append(g)
    #     mapH, apsH, mprH, prsH = utils.compute_map(ranks, gnd_t, ks)
    #     print('>> {}: mAP M: {}, H: {}'.format(args.dataset, np.around(mapM*100, decimals=2), np.around(mapH*100, decimals=2)))
    #     print('>> {}: mP@k{} M: {}, H: {}'.format(args.dataset, np.array(ks), np.around(mprM*100, decimals=2), np.around(mprH*100, decimals=2)))
    # dist.barrier()

Will run the code on one GPU.
| distributed init (rank 0): env://
git:
  sha: N/A, status: clean, branch: N/A

arch: vit_small
checkpoint_key: teacher
data_path: /content/drive/MyDrive/dataSets/oxbuild_images
dataset: roxford5k
dist_url: env://
gpu: 0
imsize: 224
local_rank: 1
multiscale: False
num_workers: 8
patch_size: 16
pretrained_weights: /content/drive/MyDrive/models/dino-main/checkpoint.pth
rank: 0
use_cuda: True
world_size: 1


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Model vit_small 16x16 built.
Take key teacher in provided checkpoint dict
Pretrained weights found at /content/drive/MyDrive/models/dino-main/checkpoint.pth and loaded with msg: _IncompatibleKeys(missing_keys=[], unexpected_keys=['head.mlp.0.weight', 'head.mlp.0.bias', 'head.mlp.2.weight', 'head.mlp.2.bias', 'head.mlp.4.weight', 'head.mlp.4.bias', 'head.last_layer.weight_g', 'head.last_layer.weight_v'])


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bicubic is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3680: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  "The default behavior for interpolate/upsample with float scale_factor changed "


Storing features into tensor of shape torch.Size([4993, 384])
  [   0/4993]  eta: 1:00:26    time: 0.726218  data: 0.644472  max mem: 91
  [  10/4993]  eta: 0:07:35    time: 0.091325  data: 0.060976  max mem: 94
  [  20/4993]  eta: 0:04:50    time: 0.024941  data: 0.002666  max mem: 94
  [  30/4993]  eta: 0:03:57    time: 0.024060  data: 0.003693  max mem: 94
  [  40/4993]  eta: 0:03:24    time: 0.023579  data: 0.003327  max mem: 94
  [  50/4993]  eta: 0:03:05    time: 0.021585  data: 0.001393  max mem: 94
  [  60/4993]  eta: 0:02:54    time: 0.022783  data: 0.001726  max mem: 94
  [  70/4993]  eta: 0:02:45    time: 0.023321  data: 0.002657  max mem: 94
  [  80/4993]  eta: 0:02:38    time: 0.022755  data: 0.002357  max mem: 94
  [  90/4993]  eta: 0:02:33    time: 0.023153  data: 0.002962  max mem: 94
  [ 100/4993]  eta: 0:02:28    time: 0.022403  data: 0.003641  max mem: 96
  [ 110/4993]  eta: 0:02:24    time: 0.021336  data: 0.003126  max mem: 96
  [ 120/4993]  eta: 0:02:21    time: 0

In [ ]:
# Copyright (c) Facebook, Inc. and its affiliates.
# 
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     http://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
import os
import sys
import pickle
import argparse

import torch
from torch import nn
import torch.distributed as dist
import torch.backends.cudnn as cudnn
from torchvision import models as torchvision_models
from torchvision import transforms as pth_transforms
from PIL import Image, ImageFile
import numpy as np
from dataset import SingleData
import utils
from utils2 import  LabelSmoothingCrossEntropyLoss, BatchHardTripletLoss, ImageReader, MPerClassSampler
from torchvision import datasets, transforms
import vision_transformer as vits
from eval_knn import extract_features
import math


def get_data_list(data_path, ratio=0.001):
    img_list = []
    for root, dirs, files in os.walk(data_path):
        if files == []:
            class_name = dirs
        elif dirs == []:
            for f in files:
                img_path = os.path.join(root, f)
                img_list.append(img_path)

    np.random.seed(1)
    train_img_list = np.random.choice(img_list, size=int(len(img_list)*(1-ratio)), replace=False)
    #print(img_list, train_img_list)
    eval_img_list = list(set(img_list) - set(train_img_list))
    ########add
    half=math.floor(len(eval_img_list)/2)
    print(half)
    eval_=eval_img_list[:half]
    test_=eval_img_list[half:]
    #######
    #return class_name, train_img_list, eval_img_list 
    return class_name, train_img_list

# class OxfordParisDataset(torch.utils.data.Dataset):
#     def __init__(self, dir_main, dataset, split, transform=None, imsize=None):
#         if dataset not in ['roxford5k', 'rparis6k']:
#             raise ValueError('Unknown dataset: {}!'.format(dataset))

#         # loading imlist, qimlist, and gnd, in cfg as a dict
#         gnd_fname = os.path.join(dir_main, dataset, 'gnd_{}.pkl'.format(dataset))
#         with open(gnd_fname, 'rb') as f:
#             cfg = pickle.load(f)
#         cfg['gnd_fname'] = gnd_fname
#         cfg['ext'] = '.jpg'
#         cfg['qext'] = '.jpg'
#         cfg['dir_data'] = os.path.join(dir_main, dataset)
#         cfg['dir_images'] = os.path.join(cfg['dir_data'], 'jpg')
#         cfg['n'] = len(cfg['imlist'])
#         cfg['nq'] = len(cfg['qimlist'])
#         cfg['im_fname'] = config_imname
#         cfg['qim_fname'] = config_qimname
#         cfg['dataset'] = dataset
#         self.cfg = cfg

#         self.samples = cfg["qimlist"] if split == "query" else cfg["imlist"]
#         self.transform = transform
#         self.imsize = imsize

#     def __len__(self):
#         return len(self.samples)

#     def __getitem__(self, index):
#         path = os.path.join(self.cfg["dir_images"], self.samples[index] + ".jpg")
#         ImageFile.LOAD_TRUNCATED_IMAGES = True
#         with open(path, 'rb') as f:
#             img = Image.open(f)
#             img = img.convert('RGB')
#         if self.imsize is not None:
#             img.thumbnail((self.imsize, self.imsize), Image.ANTIALIAS)
#         if self.transform is not None:
#             img = self.transform(img)
#         return img, index


class OxfordParisDataset(torch.utils.data.Dataset):
    def __init__(self, class_name, img_list, transform=None,imsize=None):

      self.img_list = img_list
      self.class_name = class_name
      print(self.class_name)
      self.label_list = []
      for i in range(len(img_list)):
          pre_fix = self.img_list[i].split('/')[-1].split('_')[0]
          self.label_list.append(self.class_name.index(pre_fix))
      self.label_list = np.array(self.label_list)
      self.transform = transform


      self.samples = self.img_list
      self.transform = transform
      self.imsize = imsize

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index):
        # path = os.path.join(self.cfg["dir_images"], self.samples[index] + ".jpg")
        ImageFile.LOAD_TRUNCATED_IMAGES = True
        # with open(path, 'rb') as f:
        img = Image.open(self.img_list[index])
        img = img.convert('RGB')
        if self.imsize is not None:
            img.thumbnail((self.imsize, self.imsize), Image.ANTIALIAS)
        if self.transform is not None:
            img = self.transform(img)
        return img, index




def config_imname(cfg, i):
    return os.path.join(cfg['dir_images'], cfg['imlist'][i] + cfg['ext'])


def config_qimname(cfg, i):
    return os.path.join(cfg['dir_images'], cfg['qimlist'][i] + cfg['qext'])


if __name__ == '__main__':
    parser = argparse.ArgumentParser('Image Retrieval on revisited Paris and Oxford')
    parser.add_argument('--data_path', default='/content/drive/MyDrive/dataSets/oxbuild_images', type=str)
    parser.add_argument('--dataset', default='roxford5k', type=str, choices=['roxford5k', 'rparis6k'])
    parser.add_argument('--multiscale', default=False, type=utils.bool_flag)
    parser.add_argument('--imsize', default=224, type=int, help='Image size')
    parser.add_argument('--pretrained_weights', default='/content/drive/MyDrive/models/dino-main/checkpoint.pth', type=str, help="Path to pretrained weights to evaluate.")
    parser.add_argument('--use_cuda', default=True, type=utils.bool_flag)
    parser.add_argument('--arch', default='vit_small', type=str, help='Architecture')
    parser.add_argument('--patch_size', default=16, type=int, help='Patch resolution of the model.')
    parser.add_argument("--checkpoint_key", default="teacher", type=str,
        help='Key to use in the checkpoint (example: "teacher")')
    parser.add_argument('--num_workers', default=8, type=int, help='Number of data loading workers per GPU.')
    parser.add_argument("--dist_url", default="env://", type=str, help="""url used to set up
        distributed training; see https://pytorch.org/docs/stable/distributed.html""")
    parser.add_argument("--local_rank", default=1, type=int, help="Please ignore and do not set this argument.")
    args = parser.parse_args("")

    utils.init_distributed_mode(args)
    print("git:\n  {}\n".format(utils.get_sha()))
    print("\n".join("%s: %s" % (k, str(v)) for k, v in sorted(dict(vars(args)).items())))
    cudnn.benchmark = True

    # ============ preparing data ... ============
    transform = pth_transforms.Compose([
        pth_transforms.ToTensor(),
        pth_transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])
    
    #################################################
    data_path="/content/drive/MyDrive/dataSets/CRC/DINO-test-same/train"
    # data_path="D:\\models\\CGD-master\\dataset\\img"
    test_path="/content/drive/MyDrive/dataSets/CRC/DINO-test-same/query"
    class_name, train_img_list = get_data_list(data_path,0.0)
    class_test_name, test_img_list = get_data_list(test_path,0.0)
    dataset_train=OxfordParisDataset(class_name, train_img_list, transform=transform)
    # train_sample = MPerClassSampler(train_data_set.label_list, batch_size=8)
    # data_loader_train = torch.utils.data.DataLoader(train_data_set, batch_sampler=train_sample, num_workers=8)
    # train_data_set = datasets.ImageFolder(data_path, transform=transform)
    # test_data_set = datasets.ImageFolder(test_path, transform=transform)
    dataset_query = OxfordParisDataset(class_test_name, test_img_list, transform=transform)
    # data_loader_query = torch.utils.data.DataLoader(test_data_set, batch_size=8, shuffle=False, num_workers=8)
    ##################################################
    
    # dataset_train = OxfordParisDataset(args.data_path, args.dataset, split="train", transform=transform, imsize=args.imsize)
    # dataset_query = OxfordParisDataset(args.data_path, args.dataset, split="query", transform=transform, imsize=args.imsize)
    # sampler = torch.utils.data.DistributedSampler(train_data_set, shuffle=False)
    sampler = torch.utils.data.DistributedSampler(dataset_train, shuffle=False)

    # sampler = torch.utils.data.Sampler(dataset_train)
    data_loader_train = torch.utils.data.DataLoader(
        dataset_train,
        # train_data_set,
        sampler=sampler,
        # sampler=train_sample,
        batch_size=1,
        num_workers=args.num_workers,
        pin_memory=True,
        drop_last=False,
    )
    data_loader_query = torch.utils.data.DataLoader(
        dataset_query,
        # test_data_set,
        batch_size=1,
        num_workers=args.num_workers,
        pin_memory=True,
        drop_last=False,
    )
    # print(f"train: {len(dataset_train)} imgs / query: {len(dataset_query)} imgs")

    # ============ building network ... ============
    if "vit" in args.arch:
        model = vits.__dict__[args.arch](patch_size=args.patch_size, num_classes=0)
        print(f"Model {args.arch} {args.patch_size}x{args.patch_size} built.")
    elif "xcit" in args.arch:
        model = torch.hub.load('facebookresearch/xcit:main', args.arch, num_classes=0)
    elif args.arch in torchvision_models.__dict__.keys():
        model = torchvision_models.__dict__[args.arch](num_classes=0)
    else:
        print(f"Architecture {args.arch} non supported")
        sys.exit(1)
    if args.use_cuda:
        model.cuda()
    model.eval()

    # load pretrained weights
    if os.path.isfile(args.pretrained_weights):
        state_dict = torch.load(args.pretrained_weights, map_location="cpu")
        if args.checkpoint_key is not None and args.checkpoint_key in state_dict:
            print(f"Take key {args.checkpoint_key} in provided checkpoint dict")
            state_dict = state_dict[args.checkpoint_key]
        # remove `module.` prefix
        state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}
        # remove `backbone.` prefix induced by multicrop wrapper
        state_dict = {k.replace("backbone.", ""): v for k, v in state_dict.items()}
        msg = model.load_state_dict(state_dict, strict=False)
        print('Pretrained weights found at {} and loaded with msg: {}'.format(args.pretrained_weights, msg))
    elif args.arch == "vit_small" and args.patch_size == 16:
        print("Since no pretrained weights have been provided, we load pretrained DINO weights on Google Landmark v2.")
        model.load_state_dict(torch.hub.load_state_dict_from_url(url="https://dl.fbaipublicfiles.com/dino/dino_vitsmall16_googlelandmark_pretrain/dino_vitsmall16_googlelandmark_pretrain.pth"))
    else:
        print("Warning: We use random weights.")

    ############################################################################
    # Step 1: extract features
    train_features = extract_features(model, data_loader_train, args.use_cuda, multiscale=args.multiscale)
    query_features = extract_features(model, data_loader_query, args.use_cuda, multiscale=args.multiscale)
    torch.save(train_features, 'train_features.pt')
    torch.save(query_features, 'query_features.pt')
    print(train_features)
    print(query_features)
    if utils.get_rank() == 0:  # only rank 0 will work from now on
        # normalize features
        train_features = nn.functional.normalize(train_features, dim=1, p=2)
        query_features = nn.functional.normalize(query_features, dim=1, p=2)

        ############################################################################
        # Step 2: similarity
        sim = torch.mm(train_features, query_features.T)
        ranks = torch.argsort(-sim, dim=0).cpu().numpy()
        torch.save(sim, 'sim.pt')
        print(sim)

        print(ranks)

    #     ############################################################################
    #     # Step 3: evaluate
    #     gnd = dataset_train.cfg['gnd']
    #     # evaluate ranks
    #     ks = [1, 5, 10]
    #     # search for easy & hard
    #     gnd_t = []
    #     for i in range(len(gnd)):
    #         g = {}
    #         g['ok'] = np.concatenate([gnd[i]['easy'], gnd[i]['hard']])
    #         g['junk'] = np.concatenate([gnd[i]['junk']])
    #         gnd_t.append(g)
    #     mapM, apsM, mprM, prsM = utils.compute_map(ranks, gnd_t, ks)
    #     # search for hard
    #     gnd_t = []
    #     for i in range(len(gnd)):
    #         g = {}
    #         g['ok'] = np.concatenate([gnd[i]['hard']])
    #         g['junk'] = np.concatenate([gnd[i]['junk'], gnd[i]['easy']])
    #         gnd_t.append(g)
    #     mapH, apsH, mprH, prsH = utils.compute_map(ranks, gnd_t, ks)
    #     print('>> {}: mAP M: {}, H: {}'.format(args.dataset, np.around(mapM*100, decimals=2), np.around(mapH*100, decimals=2)))
    #     print('>> {}: mP@k{} M: {}, H: {}'.format(args.dataset, np.array(ks), np.around(mprM*100, decimals=2), np.around(mprH*100, decimals=2)))
    # dist.barrier()

Will run the code on one GPU.
| distributed init (rank 0): env://


KeyboardInterrupt: ignored

In [7]:
# Copyright (c) Facebook, Inc. and its affiliates.
# 
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     http://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
import os
import sys
import pickle
import argparse

import torch
from torch import nn
import torch.distributed as dist
import torch.backends.cudnn as cudnn
from torchvision import models as torchvision_models
from torchvision import transforms as pth_transforms
from PIL import Image, ImageFile
import numpy as np
from dataset import SingleData
import utils
from utils2 import  LabelSmoothingCrossEntropyLoss, BatchHardTripletLoss, ImageReader, MPerClassSampler
from torchvision import datasets, transforms
import vision_transformer as vits
from eval_knn import extract_features
import math


def get_data_list(data_path, ratio=0.0):
    img_list = []
    for root, dirs, files in os.walk(data_path):
        if files == []:
            class_name = dirs
        elif dirs == []:
            for f in files:
                img_path = os.path.join(root, f)
                img_list.append(img_path)

    np.random.seed(1)
    train_img_list =img_list
    return class_name, train_img_list

# class OxfordParisDataset(torch.utils.data.Dataset):
#     def __init__(self, dir_main, dataset, split, transform=None, imsize=None):
#         if dataset not in ['roxford5k', 'rparis6k']:
#             raise ValueError('Unknown dataset: {}!'.format(dataset))

#         # loading imlist, qimlist, and gnd, in cfg as a dict
#         gnd_fname = os.path.join(dir_main, dataset, 'gnd_{}.pkl'.format(dataset))
#         with open(gnd_fname, 'rb') as f:
#             cfg = pickle.load(f)
#         cfg['gnd_fname'] = gnd_fname
#         cfg['ext'] = '.jpg'
#         cfg['qext'] = '.jpg'
#         cfg['dir_data'] = os.path.join(dir_main, dataset)
#         cfg['dir_images'] = os.path.join(cfg['dir_data'], 'jpg')
#         cfg['n'] = len(cfg['imlist'])
#         cfg['nq'] = len(cfg['qimlist'])
#         cfg['im_fname'] = config_imname
#         cfg['qim_fname'] = config_qimname
#         cfg['dataset'] = dataset
#         self.cfg = cfg

#         self.samples = cfg["qimlist"] if split == "query" else cfg["imlist"]
#         self.transform = transform
#         self.imsize = imsize

#     def __len__(self):
#         return len(self.samples)

#     def __getitem__(self, index):
#         path = os.path.join(self.cfg["dir_images"], self.samples[index] + ".jpg")
#         ImageFile.LOAD_TRUNCATED_IMAGES = True
#         with open(path, 'rb') as f:
#             img = Image.open(f)
#             img = img.convert('RGB')
#         if self.imsize is not None:
#             img.thumbnail((self.imsize, self.imsize), Image.ANTIALIAS)
#         if self.transform is not None:
#             img = self.transform(img)
#         return img, index


class OxfordParisDataset(torch.utils.data.Dataset):
    def __init__(self, class_name, img_list, transform=None,imsize=None):

      self.img_list = img_list
      self.class_name = class_name
      print(self.class_name)
      self.label_list = []
      for i in range(len(img_list)):
          pre_fix = self.img_list[i].split('/')[-1].split('_')[0]
          self.label_list.append(self.class_name.index(pre_fix))
      self.label_list = np.array(self.label_list)
      self.transform = transform


      self.samples = self.img_list
      self.transform = transform
      self.imsize = imsize

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index):
        # path = os.path.join(self.cfg["dir_images"], self.samples[index] + ".jpg")
        ImageFile.LOAD_TRUNCATED_IMAGES = True
        # with open(path, 'rb') as f:
        img = Image.open(self.img_list[index])
        img = img.convert('RGB')
        if self.imsize is not None:
            img.thumbnail((self.imsize, self.imsize), Image.ANTIALIAS)
        if self.transform is not None:
            img = self.transform(img)
        return img, index




def config_imname(cfg, i):
    return os.path.join(cfg['dir_images'], cfg['imlist'][i] + cfg['ext'])


def config_qimname(cfg, i):
    return os.path.join(cfg['dir_images'], cfg['qimlist'][i] + cfg['qext'])


if __name__ == '__main__':
    parser = argparse.ArgumentParser('Image Retrieval on revisited Paris and Oxford')
    parser.add_argument('--data_path', default='/content/drive/MyDrive/dataSets/oxbuild_images', type=str)
    parser.add_argument('--dataset', default='roxford5k', type=str, choices=['roxford5k', 'rparis6k'])
    parser.add_argument('--multiscale', default=False, type=utils.bool_flag)
    parser.add_argument('--imsize', default=224, type=int, help='Image size')
    # parser.add_argument('--pretrained_weights', default='/content/drive/MyDrive/models/dino-main/checkpoint 45k data-200ep/checkpoint0150.pth', type=str, help="Path to pretrained weights to evaluate.")
    # parser.add_argument('--pretrained_weights', default='/content/drive/MyDrive/models/dino-main/checkpoint 65K/checkpoint.pth', type=str, help="Path to pretrained weights to evaluate.")
    # parser.add_argument('--pretrained_weights', default='/content/drive/MyDrive/models/dino-main/checkpoint 512/checkpoint.pth', type=str, help="Path to pretrained weights to evaluate.")
    parser.add_argument('--pretrained_weights', default='/content/drive/MyDrive/models/dino-main/checkpoint.pth', type=str, help="Path to pretrained weights to evaluate.")
    # parser.add_argument('--pretrained_weights', default='/content/drive/MyDrive/models/dino-main/checkpoint-ViTB/checkpoint.pth', type=str, help="Path to pretrained weights to evaluate.")
    
    
    parser.add_argument('--use_cuda', default=True, type=utils.bool_flag)
    parser.add_argument('--arch', default='vit_small', type=str, help='Architecture')
    # parser.add_argument('--patch_size', default=16, type=int, help='Patch resolution of the model.')
    parser.add_argument('--patch_size', default=8, type=int, help='Patch resolution of the model.')
    parser.add_argument("--checkpoint_key", default="teacher", type=str,
        help='Key to use in the checkpoint (example: "teacher")')
    parser.add_argument('--num_workers', default=8, type=int, help='Number of data loading workers per GPU.')
    parser.add_argument("--dist_url", default="env://", type=str, help="""url used to set up
        distributed training; see https://pytorch.org/docs/stable/distributed.html""")
    parser.add_argument("--local_rank", default=1, type=int, help="Please ignore and do not set this argument.")
    args = parser.parse_args("")

    utils.init_distributed_mode(args)
    print("git:\n  {}\n".format(utils.get_sha()))
    print("\n".join("%s: %s" % (k, str(v)) for k, v in sorted(dict(vars(args)).items())))
    cudnn.benchmark = True

    # ============ preparing data ... ============
    transform = pth_transforms.Compose([
        pth_transforms.ToTensor(),
        pth_transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])
    
    #################################################
    # data_path="/content/drive/MyDrive/dataSets/CRC/DINO-test-same/train"
    # data_path="/content/drive/MyDrive/dataSets/CRC/forDINO/train"
    data_path="/content/drive/MyDrive/dataSets/test_bracs"
   
    # data_path="D:\\models\\CGD-master\\dataset\\img"
    # test_path="/content/drive/MyDrive/dataSets/CRC/fixed different test2"
    test_path="/content/drive/MyDrive/dataSets/test_bracs"

    # test_path="/content/drive/MyDrive/dataSets/CRC/fixed same test"
    class_name, train_img_list = get_data_list(data_path,0.0)
    class_test_name, test_img_list = get_data_list(test_path,0.0)
    dataset_train=OxfordParisDataset(class_name, train_img_list, transform=transform)
    # train_sample = MPerClassSampler(train_data_set.label_list, batch_size=8)
    # data_loader_train = torch.utils.data.DataLoader(train_data_set, batch_sampler=train_sample, num_workers=8)
    # train_data_set = datasets.ImageFolder(data_path, transform=transform)
    # test_data_set = datasets.ImageFolder(test_path, transform=transform)
    dataset_query = OxfordParisDataset(class_test_name, test_img_list, transform=transform)
    # data_loader_query = torch.utils.data.DataLoader(test_data_set, batch_size=8, shuffle=False, num_workers=8)
    ##################################################
    
    # dataset_train = OxfordParisDataset(args.data_path, args.dataset, split="train", transform=transform, imsize=args.imsize)
    # dataset_query = OxfordParisDataset(args.data_path, args.dataset, split="query", transform=transform, imsize=args.imsize)
    # sampler = torch.utils.data.DistributedSampler(train_data_set, shuffle=False)
    sampler = torch.utils.data.DistributedSampler(dataset_train, shuffle=False)

    # sampler = torch.utils.data.Sampler(dataset_train)
    data_loader_train = torch.utils.data.DataLoader(
        dataset_train,
        # train_data_set,
        sampler=sampler,
        # sampler=train_sample,
        batch_size=1,
        num_workers=args.num_workers,
        pin_memory=True,
        drop_last=False,
    )
    data_loader_query = torch.utils.data.DataLoader(
        dataset_query,
        # test_data_set,
        batch_size=1,
        num_workers=args.num_workers,
        pin_memory=True,
        drop_last=False,
    )
    # print(f"train: {len(dataset_train)} imgs / query: {len(dataset_query)} imgs")
    # print(data_loader_train.dataset.samples)
    # ============ building network ... ============
    if "vit" in args.arch:
        model = vits.__dict__[args.arch](patch_size=args.patch_size, num_classes=0)
        print(f"Model {args.arch} {args.patch_size}x{args.patch_size} built.")
    elif "xcit" in args.arch:
        model = torch.hub.load('facebookresearch/xcit:main', args.arch, num_classes=0)
    elif args.arch in torchvision_models.__dict__.keys():
        model = torchvision_models.__dict__[args.arch](num_classes=0)
    else:
        print(f"Architecture {args.arch} non supported")
        sys.exit(1)
    if args.use_cuda:
        model.cuda()
    model.eval()

    # load pretrained weights
    if os.path.isfile(args.pretrained_weights):
        state_dict = torch.load(args.pretrained_weights, map_location="cpu")
        if args.checkpoint_key is not None and args.checkpoint_key in state_dict:
            print(f"Take key {args.checkpoint_key} in provided checkpoint dict")
            state_dict = state_dict[args.checkpoint_key]
        # remove `module.` prefix
        state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}
        # remove `backbone.` prefix induced by multicrop wrapper
        state_dict = {k.replace("backbone.", ""): v for k, v in state_dict.items()}
        msg = model.load_state_dict(state_dict, strict=False)
        print('Pretrained weights found at {} and loaded with msg: {}'.format(args.pretrained_weights, msg))
    elif args.arch == "vit_small" and args.patch_size == 16:
        print("Since no pretrained weights have been provided, we load pretrained DINO weights on Google Landmark v2.")
        model.load_state_dict(torch.hub.load_state_dict_from_url(url="https://dl.fbaipublicfiles.com/dino/dino_vitsmall16_googlelandmark_pretrain/dino_vitsmall16_googlelandmark_pretrain.pth"))
    else:
        print("Warning: We use random weights.")

    ############################################################################
    # Step 1: extract features
    print("###########",data_loader_query.dataset.img_list)
    torch.save(data_loader_query.dataset.img_list, 'img_names.pt')
    train_features,train_label = extract_features(model, data_loader_train, args.use_cuda, multiscale=args.multiscale)
    query_features,query_label = extract_features(model, data_loader_query, args.use_cuda, multiscale=args.multiscale)
    torch.save(train_features, 'train_features.pt')
    torch.save(query_features, 'query_features.pt')
    torch.save(train_label, 'train_label.pt')
    torch.save(query_label, 'query_label.pt')
    

    
    
    print(train_features)
    print(query_features)
    if utils.get_rank() == 0:  # only rank 0 will work from now on
        # normalize features
        train_features = nn.functional.normalize(train_features, dim=1, p=2)
        query_features = nn.functional.normalize(query_features, dim=1, p=2)

        ############################################################################
        # Step 2: similarity
        sim = torch.mm(train_features, query_features.T)
        ranks = torch.argsort(-sim, dim=0).cpu().numpy()
        torch.save(sim, 'sim.pt')
        print(sim)

        print(ranks)

    #     ############################################################################
    #     # Step 3: evaluate
    #     gnd = dataset_train.cfg['gnd']
    #     # evaluate ranks
    #     ks = [1, 5, 10]
    #     # search for easy & hard
    #     gnd_t = []
    #     for i in range(len(gnd)):
    #         g = {}
    #         g['ok'] = np.concatenate([gnd[i]['easy'], gnd[i]['hard']])
    #         g['junk'] = np.concatenate([gnd[i]['junk']])
    #         gnd_t.append(g)
    #     mapM, apsM, mprM, prsM = utils.compute_map(ranks, gnd_t, ks)
    #     # search for hard
    #     gnd_t = []
    #     for i in range(len(gnd)):
    #         g = {}
    #         g['ok'] = np.concatenate([gnd[i]['hard']])
    #         g['junk'] = np.concatenate([gnd[i]['junk'], gnd[i]['easy']])
    #         gnd_t.append(g)
    #     mapH, apsH, mprH, prsH = utils.compute_map(ranks, gnd_t, ks)
    #     print('>> {}: mAP M: {}, H: {}'.format(args.dataset, np.around(mapM*100, decimals=2), np.around(mapH*100, decimals=2)))
    #     print('>> {}: mP@k{} M: {}, H: {}'.format(args.dataset, np.array(ks), np.around(mprM*100, decimals=2), np.around(mprH*100, decimals=2)))
    # dist.barrier()

Will run the code on one GPU.
| distributed init (rank 0): env://
git:
  sha: N/A, status: clean, branch: N/A

arch: vit_small
checkpoint_key: teacher
data_path: /content/drive/MyDrive/dataSets/oxbuild_images
dataset: roxford5k
dist_url: env://
gpu: 0
imsize: 224
local_rank: 1
multiscale: False
num_workers: 8
patch_size: 8
pretrained_weights: /content/drive/MyDrive/models/dino-main/checkpoint.pth
rank: 0
use_cuda: True
world_size: 1
['s0', 's1', 's2', 's3', 's4', 's5', 's6']
['s0', 's1', 's2', 's3', 's4', 's5', 's6']


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Model vit_small 8x8 built.
Take key teacher in provided checkpoint dict
Pretrained weights found at /content/drive/MyDrive/models/dino-main/checkpoint.pth and loaded with msg: _IncompatibleKeys(missing_keys=[], unexpected_keys=['head.mlp.0.weight', 'head.mlp.0.bias', 'head.mlp.2.weight', 'head.mlp.2.bias', 'head.mlp.4.weight', 'head.mlp.4.bias', 'head.last_layer.weight_g', 'head.last_layer.weight_v'])
########### ['/content/drive/MyDrive/dataSets/test_bracs/s0/s0_1078_s0_2_100.png', '/content/drive/MyDrive/dataSets/test_bracs/s0/s0_864_s0_1_8.png', '/content/drive/MyDrive/dataSets/test_bracs/s0/s0_1985_s0_5_148.png', '/content/drive/MyDrive/dataSets/test_bracs/s0/s0_2800_s0_8_148.png', '/content/drive/MyDrive/dataSets/test_bracs/s0/s0_1066_s0_2_100.png', '/content/drive/MyDrive/dataSets/test_bracs/s0/s0_4_s0_0_100.png', '/content/drive/MyDrive/dataSets/test_bracs/s0/s0_1062_s0_2_100.png', '/content/drive/MyDrive/dataSets/test_bracs/s0/s0_1662_s0_3_148.png', '/content/drive/MyDrive/data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pwd 
%cd drive/MyDrive/models/dino-main/

Mounted at /content/drive
/content
/content/drive/MyDrive/models/dino-main


15 155
0 2 520 159
0 6 240 126
0 3 940 220
0 13 1810 1
0 7 2400 14
0 10 820 185
0 4 1 66
0 12 720 127
0 11 1 85
0 1 1270 167
0 8 820 61
0 14 1 253
0 9 1530 36
1 6 193 233
1 8 191 253
1 13 1 24
1 3 139 131
1 7 226 78
1 12 1 14
1 11 50 1
1 4 87 229
1 2 277 31
1 1 34 82
2 6 1 1
2 9 159 204
2 7 205 1
2 2 1 7
2 3 1 79
2 5 145 186
2 11 161 1
2 4 1 38
2 13 92 74
2 14 1530 81
2 8 97 155
3 8 15 1
3 3 225 183
3 6 115 1
3 12 1 5
3 13 147 189
3 14 1 1
3 2 113 92
3 5 218 212
3 10 288 242
3 9 49 159
3 4 80 40
3 7 151 207
3 1 282 161


test for vitB

In [ ]:
# Copyright (c) Facebook, Inc. and its affiliates.
# 
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     http://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
import os
import sys
import pickle
import argparse

import torch
from torch import nn
import torch.distributed as dist
import torch.backends.cudnn as cudnn
from torchvision import models as torchvision_models
from torchvision import transforms as pth_transforms
from PIL import Image, ImageFile
import numpy as np
from dataset import SingleData
import utils
from utils2 import  LabelSmoothingCrossEntropyLoss, BatchHardTripletLoss, ImageReader, MPerClassSampler
from torchvision import datasets, transforms
import vision_transformer as vits
from eval_knn import extract_features
import math


def get_data_list(data_path, ratio=0.0):
    img_list = []
    for root, dirs, files in os.walk(data_path):
        if files == []:
            class_name = dirs
        elif dirs == []:
            for f in files:
                img_path = os.path.join(root, f)
                img_list.append(img_path)

    np.random.seed(1)
    # train_img_list = np.random.choice(img_list, size=int(len(img_list)*(1-ratio)), replace=False)
    train_img_list =img_list
    #print(img_list, train_img_list)
    # eval_img_list = list(set(img_list) - set(train_img_list))
    ########add
    # half=math.floor(len(eval_img_list)/2)
    # print(half)
    # eval_=eval_img_list[:half]
    # test_=eval_img_list[half:]
    #######
    #return class_name, train_img_list, eval_img_list 
    return class_name, train_img_list

# class OxfordParisDataset(torch.utils.data.Dataset):
#     def __init__(self, dir_main, dataset, split, transform=None, imsize=None):
#         if dataset not in ['roxford5k', 'rparis6k']:
#             raise ValueError('Unknown dataset: {}!'.format(dataset))

#         # loading imlist, qimlist, and gnd, in cfg as a dict
#         gnd_fname = os.path.join(dir_main, dataset, 'gnd_{}.pkl'.format(dataset))
#         with open(gnd_fname, 'rb') as f:
#             cfg = pickle.load(f)
#         cfg['gnd_fname'] = gnd_fname
#         cfg['ext'] = '.jpg'
#         cfg['qext'] = '.jpg'
#         cfg['dir_data'] = os.path.join(dir_main, dataset)
#         cfg['dir_images'] = os.path.join(cfg['dir_data'], 'jpg')
#         cfg['n'] = len(cfg['imlist'])
#         cfg['nq'] = len(cfg['qimlist'])
#         cfg['im_fname'] = config_imname
#         cfg['qim_fname'] = config_qimname
#         cfg['dataset'] = dataset
#         self.cfg = cfg

#         self.samples = cfg["qimlist"] if split == "query" else cfg["imlist"]
#         self.transform = transform
#         self.imsize = imsize

#     def __len__(self):
#         return len(self.samples)

#     def __getitem__(self, index):
#         path = os.path.join(self.cfg["dir_images"], self.samples[index] + ".jpg")
#         ImageFile.LOAD_TRUNCATED_IMAGES = True
#         with open(path, 'rb') as f:
#             img = Image.open(f)
#             img = img.convert('RGB')
#         if self.imsize is not None:
#             img.thumbnail((self.imsize, self.imsize), Image.ANTIALIAS)
#         if self.transform is not None:
#             img = self.transform(img)
#         return img, index


class OxfordParisDataset(torch.utils.data.Dataset):
    def __init__(self, class_name, img_list, transform=None,imsize=None):

      self.img_list = img_list
      self.class_name = class_name
      print(self.class_name)
      self.label_list = []
      for i in range(len(img_list)):
          pre_fix = self.img_list[i].split('/')[-1].split('_')[0]
          self.label_list.append(self.class_name.index(pre_fix))
      self.label_list = np.array(self.label_list)
      self.transform = transform


      self.samples = self.img_list
      self.transform = transform
      self.imsize = imsize

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index):
        # path = os.path.join(self.cfg["dir_images"], self.samples[index] + ".jpg")
        ImageFile.LOAD_TRUNCATED_IMAGES = True
        # with open(path, 'rb') as f:
        img = Image.open(self.img_list[index])
        img = img.convert('RGB')
        if self.imsize is not None:
            img.thumbnail((self.imsize, self.imsize), Image.ANTIALIAS)
        if self.transform is not None:
            img = self.transform(img)
        return img, index




def config_imname(cfg, i):
    return os.path.join(cfg['dir_images'], cfg['imlist'][i] + cfg['ext'])


def config_qimname(cfg, i):
    return os.path.join(cfg['dir_images'], cfg['qimlist'][i] + cfg['qext'])


if __name__ == '__main__':
    parser = argparse.ArgumentParser('Image Retrieval on revisited Paris and Oxford')
    parser.add_argument('--data_path', default='/content/drive/MyDrive/dataSets/oxbuild_images', type=str)
    parser.add_argument('--dataset', default='roxford5k', type=str, choices=['roxford5k', 'rparis6k'])
    parser.add_argument('--multiscale', default=False, type=utils.bool_flag)
    parser.add_argument('--imsize', default=224, type=int, help='Image size')
    # parser.add_argument('--pretrained_weights', default='/content/drive/MyDrive/models/dino-main/checkpoint 45k data-200ep/checkpoint.pth', type=str, help="Path to pretrained weights to evaluate.")
    # parser.add_argument('--pretrained_weights', default='/content/drive/MyDrive/models/dino-main/checkpoint 65K/checkpoint.pth', type=str, help="Path to pretrained weights to evaluate.")
    # parser.add_argument('--pretrained_weights', default='/content/drive/MyDrive/models/dino-main/checkpoint 512/checkpoint.pth', type=str, help="Path to pretrained weights to evaluate.")
    parser.add_argument('--pretrained_weights', default='/content/drive/MyDrive/models/dino-main/checkpoint.pth', type=str, help="Path to pretrained weights to evaluate.")
    # parser.add_argument('--pretrained_weights', default='/content/drive/MyDrive/models/dino-main/checkpoint-ViTB/checkpoint.pth', type=str, help="Path to pretrained weights to evaluate.")
    
    
    parser.add_argument('--use_cuda', default=True, type=utils.bool_flag)
    parser.add_argument('--arch', default='vit_base', type=str, help='Architecture')
    parser.add_argument('--patch_size', default=16, type=int, help='Patch resolution of the model.')
    parser.add_argument("--checkpoint_key", default="teacher", type=str,
        help='Key to use in the checkpoint (example: "teacher")')
    parser.add_argument('--num_workers', default=8, type=int, help='Number of data loading workers per GPU.')
    parser.add_argument("--dist_url", default="env://", type=str, help="""url used to set up
        distributed training; see https://pytorch.org/docs/stable/distributed.html""")
    parser.add_argument("--local_rank", default=1, type=int, help="Please ignore and do not set this argument.")
    args = parser.parse_args("")

    utils.init_distributed_mode(args)
    print("git:\n  {}\n".format(utils.get_sha()))
    print("\n".join("%s: %s" % (k, str(v)) for k, v in sorted(dict(vars(args)).items())))
    cudnn.benchmark = True

    # ============ preparing data ... ============
    transform = pth_transforms.Compose([
        pth_transforms.ToTensor(),
        pth_transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])
    
    #################################################
    # data_path="/content/drive/MyDrive/dataSets/CRC/forDINO/train"
    data_path="/content/drive/MyDrive/dataSets/CRC/forDINO/val/fixed-1000"
    # data_path="/content/drive/MyDrive/dataSets/CRC/for DINO/train/"
   
    # data_path="D:\\models\\CGD-master\\dataset\\img"
    # test_path="/content/drive/MyDrive/dataSets/CRC/fixed different test2"
    #test_path data_path="/content/drive/MyDrive/dataSets/CRC/forDINO/train"
    test_path="/content/drive/MyDrive/dataSets/CRC/forDINO/val/fixed-1000"

    # test_path="/content/drive/MyDrive/dataSets/CRC/fixed same test"
    class_name, train_img_list = get_data_list(data_path,0.0)
    class_test_name, test_img_list = get_data_list(test_path,0.0)
    dataset_train=OxfordParisDataset(class_name, train_img_list, transform=transform)
    # train_sample = MPerClassSampler(train_data_set.label_list, batch_size=8)
    # data_loader_train = torch.utils.data.DataLoader(train_data_set, batch_sampler=train_sample, num_workers=8)
    # train_data_set = datasets.ImageFolder(data_path, transform=transform)
    # test_data_set = datasets.ImageFolder(test_path, transform=transform)
    dataset_query = OxfordParisDataset(class_test_name, test_img_list, transform=transform)
    # data_loader_query = torch.utils.data.DataLoader(test_data_set, batch_size=8, shuffle=False, num_workers=8)
    ##################################################
    
    # dataset_train = OxfordParisDataset(args.data_path, args.dataset, split="train", transform=transform, imsize=args.imsize)
    # dataset_query = OxfordParisDataset(args.data_path, args.dataset, split="query", transform=transform, imsize=args.imsize)
    # sampler = torch.utils.data.DistributedSampler(train_data_set, shuffle=False)
    sampler = torch.utils.data.DistributedSampler(dataset_train, shuffle=False)

    # sampler = torch.utils.data.Sampler(dataset_train)
    data_loader_train = torch.utils.data.DataLoader(
        dataset_train,
        # train_data_set,
        sampler=sampler,
        # sampler=train_sample,
        batch_size=1,
        num_workers=args.num_workers,
        pin_memory=True,
        drop_last=False,
    )
    data_loader_query = torch.utils.data.DataLoader(
        dataset_query,
        # test_data_set,
        batch_size=1,
        num_workers=args.num_workers,
        pin_memory=True,
        drop_last=False,
    )
    # print(f"train: {len(dataset_train)} imgs / query: {len(dataset_query)} imgs")
    # print(data_loader_train.dataset.samples)
    # ============ building network ... ============
    if "vit" in args.arch:
        model = vits.__dict__[args.arch](patch_size=args.patch_size, num_classes=0)
        print(f"Model {args.arch} {args.patch_size}x{args.patch_size} built.")
    elif "xcit" in args.arch:
        model = torch.hub.load('facebookresearch/xcit:main', args.arch, num_classes=0)
    elif args.arch in torchvision_models.__dict__.keys():
        model = torchvision_models.__dict__[args.arch](num_classes=0)
    else:
        print(f"Architecture {args.arch} non supported")
        sys.exit(1)
    if args.use_cuda:
        model.cuda()
    model.eval()

    # load pretrained weights
    if os.path.isfile(args.pretrained_weights):
        state_dict = torch.load(args.pretrained_weights, map_location="cpu")
        if args.checkpoint_key is not None and args.checkpoint_key in state_dict:
            print(f"Take key {args.checkpoint_key} in provided checkpoint dict")
            state_dict = state_dict[args.checkpoint_key]
        # remove `module.` prefix
        state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}
        # remove `backbone.` prefix induced by multicrop wrapper
        state_dict = {k.replace("backbone.", ""): v for k, v in state_dict.items()}
        msg = model.load_state_dict(state_dict, strict=False)
        print('Pretrained weights found at {} and loaded with msg: {}'.format(args.pretrained_weights, msg))
    elif args.arch == "vit_small" and args.patch_size == 16:
        print("Since no pretrained weights have been provided, we load pretrained DINO weights on Google Landmark v2.")
        model.load_state_dict(torch.hub.load_state_dict_from_url(url="https://dl.fbaipublicfiles.com/dino/dino_vitsmall16_googlelandmark_pretrain/dino_vitsmall16_googlelandmark_pretrain.pth"))
    else:
        print("Warning: We use random weights.")

    ############################################################################
    # Step 1: extract features
    print("###########",data_loader_query.dataset.img_list)
    torch.save(data_loader_query.dataset.img_list, 'img_names.pt')
    train_features,train_label = extract_features(model, data_loader_train, args.use_cuda, multiscale=args.multiscale)
    query_features,query_label = extract_features(model, data_loader_query, args.use_cuda, multiscale=args.multiscale)
    torch.save(train_features, 'train_features.pt')
    torch.save(query_features, 'query_features.pt')
    torch.save(train_label, 'train_label.pt')
    torch.save(query_label, 'query_label.pt')
    

    
    
    print(train_features)
    print(query_features)
    if utils.get_rank() == 0:  # only rank 0 will work from now on
        # normalize features
        train_features = nn.functional.normalize(train_features, dim=1, p=2)
        query_features = nn.functional.normalize(query_features, dim=1, p=2)

        ############################################################################
        # Step 2: similarity
        sim = torch.mm(train_features, query_features.T)
        ranks = torch.argsort(-sim, dim=0).cpu().numpy()
        torch.save(sim, 'sim.pt')
        print(sim)

        print(ranks)

    #     ############################################################################
    #     # Step 3: evaluate
    #     gnd = dataset_train.cfg['gnd']
    #     # evaluate ranks
    #     ks = [1, 5, 10]
    #     # search for easy & hard
    #     gnd_t = []
    #     for i in range(len(gnd)):
    #         g = {}
    #         g['ok'] = np.concatenate([gnd[i]['easy'], gnd[i]['hard']])
    #         g['junk'] = np.concatenate([gnd[i]['junk']])
    #         gnd_t.append(g)
    #     mapM, apsM, mprM, prsM = utils.compute_map(ranks, gnd_t, ks)
    #     # search for hard
    #     gnd_t = []
    #     for i in range(len(gnd)):
    #         g = {}
    #         g['ok'] = np.concatenate([gnd[i]['hard']])
    #         g['junk'] = np.concatenate([gnd[i]['junk'], gnd[i]['easy']])
    #         gnd_t.append(g)
    #     mapH, apsH, mprH, prsH = utils.compute_map(ranks, gnd_t, ks)
    #     print('>> {}: mAP M: {}, H: {}'.format(args.dataset, np.around(mapM*100, decimals=2), np.around(mapH*100, decimals=2)))
    #     print('>> {}: mP@k{} M: {}, H: {}'.format(args.dataset, np.array(ks), np.around(mprM*100, decimals=2), np.around(mprH*100, decimals=2)))
    # dist.barrier()

Will run the code on one GPU.
| distributed init (rank 0): env://
git:
  sha: N/A, status: clean, branch: N/A

arch: vit_base
checkpoint_key: teacher
data_path: /content/drive/MyDrive/dataSets/oxbuild_images
dataset: roxford5k
dist_url: env://
gpu: 0
imsize: 224
local_rank: 1
multiscale: False
num_workers: 8
patch_size: 16
pretrained_weights: /content/drive/MyDrive/models/dino-main/checkpoint.pth
rank: 0
use_cuda: True
world_size: 1
['s3', 's2', 's1']
['s3', 's2', 's1']


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Model vit_base 16x16 built.
Take key teacher in provided checkpoint dict


RuntimeError: ignored

In [ ]:

import vision_transformer as vits
vits.__dict__.keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__file__', '__cached__', '__builtins__', 'math', 'partial', 'torch', 'nn', 'trunc_normal_', 'drop_path', 'DropPath', 'Mlp', 'Attention', 'Block', 'PatchEmbed', 'VisionTransformer', 'vit_tiny', 'vit_small', 'vit_base', 'DINOHead'])